In [1]:
import tensorflow as tf
print('TensorFlow Version: {}'.format(tf.__version__))
print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.7.1
Default GPU Device: 


##### >**Note:** Make sure you have OpenAI Gym cloned. Then run this command `pip install -e gym/[all]`.

In [2]:
import numpy as np
import gym
env = gym.make('CartPole-v0')
env = gym.make('CartPole-v1')

In [3]:
def model_input(state_size):
    #states = tf.placeholder(tf.float32, [None, *state_size], name='states')
    states = tf.placeholder(tf.float32, [None, state_size], name='states')
    actions = tf.placeholder(tf.int32, [None], name='actions')
    next_states = tf.placeholder(tf.float32, [None, state_size], name='next_states')
    rewards = tf.placeholder(tf.float32, [None], name='rewards')
    dones = tf.placeholder(tf.float32, [None], name='dones')
    rates = tf.placeholder(tf.float32, [None], name='rates') # success rate
    return states, actions, next_states, rewards, dones, rates

In [4]:
def Act(states, action_size, hidden_size, reuse=False, alpha=0.1, training=False):
    with tf.variable_scope('Act', reuse=reuse):
        # First fully connected layer
        h1 = tf.layers.dense(inputs=states, units=hidden_size)
        bn1 = tf.layers.batch_normalization(h1, training=training)        
        nl1 = tf.maximum(alpha * bn1, bn1)
        
        # Second fully connected layer
        h2 = tf.layers.dense(inputs=nl1, units=hidden_size)
        bn2 = tf.layers.batch_normalization(h2, training=training)        
        nl2 = tf.maximum(alpha * bn2, bn2)
        
        # Output layer
        logits = tf.layers.dense(inputs=nl2, units=action_size)        
        return logits

In [5]:
def Env(states, actions, state_size, action_size, hidden_size, reuse=False, alpha=0.1, training=False):
    with tf.variable_scope('Env', reuse=reuse):
        # First fully connected layer
        h1 = tf.layers.dense(inputs=states, units=action_size)
        bn1 = tf.layers.batch_normalization(h1, training=training)        
        nl1 = tf.maximum(alpha * bn1, bn1)
        
        # Second fully connected layer
        nl1_fused = tf.concat(axis=1, values=[nl1, actions])
        h2 = tf.layers.dense(inputs=nl1_fused, units=hidden_size)
        bn2 = tf.layers.batch_normalization(h2, training=training)        
        nl2 = tf.maximum(alpha * bn2, bn2)
                
        # Output layer
        states_logits = tf.layers.dense(inputs=nl2, units=state_size, trainable=False)
        Qlogits = tf.layers.dense(inputs=nl2, units=1, trainable=False)
        return states_logits, Qlogits

In [14]:
def model_loss(state_size, action_size, hidden_size, gamma,
               states, actions, next_states, rewards, dones, rates):
    ################################################ a = act(s)
    actions_logits = Act(states=states, hidden_size=hidden_size, action_size=action_size)
    actions_labels = tf.one_hot(indices=actions, depth=action_size, dtype=actions_logits.dtype)
    aloss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=actions_logits, 
                                                                      labels=actions_labels))
    ################################################ s', r = env(s, a)
    e_next_states_logits, eQs = Env(actions=actions_labels, states=states, hidden_size=hidden_size, 
                                    action_size=action_size, state_size=state_size)
    a_next_states_logits, aQs = Env(actions=actions_logits, states=states, hidden_size=hidden_size, 
                                    action_size=action_size, state_size=state_size, reuse=True)
    next_states_labels = tf.nn.sigmoid(next_states)
    eloss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=e_next_states_logits, # GQN
                                                                   labels=next_states_labels))
    eloss += -tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=a_next_states_logits, 
                                                                     labels=next_states_labels)) # maximize loss
    aloss2 = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=a_next_states_logits, 
                                                                    labels=next_states_labels)) # minimize loss
    eQs_logits = tf.reshape(eQs, shape=[-1])
    aQs_logits = tf.reshape(aQs, shape=[-1])
    #################################################### s'', Q' = ~env(s', ~a')
    next_actions_logits = Act(states=next_states, hidden_size=hidden_size, action_size=action_size, reuse=True)
    next_states_logits, aQs2 = Env(actions=next_actions_logits, states=next_states, hidden_size=hidden_size, 
                                   action_size=action_size, state_size=state_size, reuse=True)
    aQs2_logits = tf.reshape(aQs2, shape=[-1]) * (1-dones)
    eloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=eQs_logits, # GAN
                                                                    labels=rates)) # 0-1 real
#     eloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=(aQs_logits+aQs2_logits)/2, # GAN
#                                                                     labels=tf.zeros_like(rates))) # min
#     aloss2 += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=(aQs_logits+aQs2_logits)/2, # GAN
#                                                                      labels=tf.ones_like(rates))) # max
    ###################################################### Q(s,a)= r + Q'(s',a')
    targetQs = rewards + (gamma * aQs2_logits) # DQN/Qlearning
    eloss += tf.reduce_mean(tf.square(eQs_logits - targetQs)) # real
#     eloss += tf.reduce_mean(tf.square(aQs_logits - rewards)) # min
#     aloss2 += tf.reduce_mean(tf.square(aQs_logits - targetQs)) # max
    eloss += tf.reduce_mean((aQs_logits+aQs2_logits)/2) # min
    aloss2 += -tf.reduce_mean((aQs_logits+aQs2_logits)/2) # max
    #################################################### s'', Q' = ~env(s', ~a'): Repeatable 1
    next_actions_logits = Act(states=next_states_logits, hidden_size=hidden_size, action_size=action_size, 
                              reuse=True)
    next_states_logits, aQs2 = Env(actions=next_actions_logits, states=next_states_logits, hidden_size=hidden_size, 
                                   action_size=action_size, state_size=state_size, reuse=True)
    aQs_logits = aQs2_logits # pervious one
    aQs2_logits = tf.reshape(aQs2, shape=[-1]) * (1-dones)
#     eloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=(aQs_logits+aQs2_logits)/2, # GAN
#                                                                     labels=tf.zeros_like(rates))) # min
#     aloss2 += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=(aQs_logits+aQs2_logits)/2, # GAN
#                                                                      labels=tf.ones_like(rates))) # max
    ###################################################### Q(s,a)= r + Q'(s',a')
    targetQs = rewards + (gamma * aQs2_logits) # DQN/Qlearning
    eloss += tf.reduce_mean(tf.square(aQs_logits - rewards)) # min
    aloss2 += tf.reduce_mean(tf.square(aQs_logits - targetQs)) # max
    #################################################### s'', Q' = ~env(s', ~a'): Repeatable 2
    next_actions_logits = Act(states=next_states_logits, hidden_size=hidden_size, action_size=action_size, 
                              reuse=True)
    next_states_logits, aQs2 = Env(actions=next_actions_logits, states=next_states_logits, hidden_size=hidden_size, 
                                   action_size=action_size, state_size=state_size, reuse=True)
    aQs_logits = aQs2_logits # pervious one
    aQs2_logits = tf.reshape(aQs2, shape=[-1]) * (1-dones)
#     eloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=(aQs_logits+aQs2_logits)/2, # GAN
#                                                                     labels=tf.zeros_like(rates))) # min
#     aloss2 += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=(aQs_logits+aQs2_logits)/2, # GAN
#                                                                      labels=tf.ones_like(rates))) # max
    ###################################################### Q(s,a)= r + Q'(s',a')
    targetQs = rewards + (gamma * aQs2_logits) # DQN/Qlearning
    eloss += tf.reduce_mean(tf.square(aQs_logits - rewards)) # min
    aloss2 += tf.reduce_mean(tf.square(aQs_logits - targetQs)) # max
    return actions_logits, aloss, eloss, aloss2

In [15]:
def model_opt(a_loss, e_loss, a_loss2, a_learning_rate, e_learning_rate):
    # Get weights and bias to update
    t_vars = tf.trainable_variables()
    a_vars = [var for var in t_vars if var.name.startswith('Act')]
    e_vars = [var for var in t_vars if var.name.startswith('Env')]

    # Optimize
    with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)): # Required for batchnorm (BN)
        a_opt = tf.train.AdamOptimizer(a_learning_rate).minimize(a_loss, var_list=a_vars)
        e_opt = tf.train.AdamOptimizer(e_learning_rate).minimize(e_loss, var_list=e_vars)
        a_opt2 = tf.train.AdamOptimizer(a_learning_rate).minimize(a_loss2, var_list=a_vars)
    return a_opt, e_opt, a_opt2

In [16]:
class Model:
    def __init__(self, state_size, action_size, hidden_size, a_learning_rate, e_learning_rate, gamma):

        # Data of the Model: make the data available inside the framework
        self.states, self.actions, self.next_states, self.rewards, self.dones, self.rates = model_input(
            state_size=state_size)

        # Create the Model: calculating the loss and forwad pass
        self.actions_logits, self.a_loss, self.e_loss, self.a_loss2 = model_loss(
            state_size=state_size, action_size=action_size, hidden_size=hidden_size, gamma=gamma, # model init
            states=self.states, actions=self.actions, next_states=self.next_states, 
            rewards=self.rewards, dones=self.dones, rates=self.rates) # model input
        
        # Update the model: backward pass and backprop
        self.a_opt, self.e_opt, self.a_opt2 = model_opt(a_loss=self.a_loss, 
                                                        e_loss=self.e_loss,
                                                        a_loss2=self.a_loss2, 
                                                        a_learning_rate=a_learning_rate,
                                                        e_learning_rate=e_learning_rate)

In [17]:
from collections import deque
class Memory():
    def __init__(self, max_size = 1000):
        self.buffer = deque(maxlen=max_size) # data batch
#     def sample(self, batch_size):
#         idx = np.random.choice(np.arange(len(self.buffer)), size=batch_size, replace=False)
#         return [self.buffer[ii] for ii in idx]

## Hyperparameters

One of the more difficult aspects of reinforcememt learning are the large number of hyperparameters. Not only are we tuning the network, but we're tuning the simulation.

In [18]:
env.observation_space, env.action_space

(Box(4,), Discrete(2))

In [19]:
# Exploration parameters
explore_start = 1.0            # exploration probability at start
explore_stop = 0.01           # minimum exploration probability 
decay_rate = 0.0001            # exponential decay rate for exploration prob

# Network parameters
state_size = 4
action_size = 2
hidden_size = 4*2             # number of units in each Q-network hidden layer
a_learning_rate = 1e-4         # Q-network learning rate
e_learning_rate = 1e-4         # Q-network learning rate

# Memory parameters
memory_size = int(1e5)            # memory capacity
batch_size = int(1e3)             # experience mini-batch size
gamma=0.99

In [20]:
# Reset/init the graph/session
graph = tf.reset_default_graph()

# Init the model
model = Model(action_size=action_size, state_size=state_size, hidden_size=hidden_size, gamma=gamma,
              a_learning_rate=a_learning_rate, e_learning_rate=e_learning_rate)

# Init the memory
memory = Memory(max_size=memory_size)

In [21]:
state = env.reset()
total_reward = 0
num_step = 0
for _ in range(memory_size):
    action = env.action_space.sample()
    next_state, reward, done, _ = env.step(action)
    rate = -1
    memory.buffer.append([state, action, next_state, reward, float(done), rate])
    num_step += 1 # memory incremented
    total_reward += reward
    state = next_state
    if done is True:
        rate = total_reward/500
        for idx in range(num_step): # episode length
            if memory.buffer[-1-idx][-1] == -1:
                memory.buffer[-1-idx][-1] = rate
        state = env.reset()
        total_reward = 0 # reset
        num_step = 0 # reset

## Training the model

Below we'll train our agent. If you want to watch it train, uncomment the `env.render()` line. This is slow because it's rendering the frames slower than the network can train. But, it's cool to watch the agent get better at the game.

In [ ]:
# Save/load the model and save for plotting
saver = tf.train.Saver()
episode_rewards_list, rewards_list = [], []
aloss_list, eloss_list, aloss2_list = [], [], []

# TF session for training
with tf.Session(graph=graph) as sess:
    sess.run(tf.global_variables_initializer())
    #saver.restore(sess, 'checkpoints/model.ckpt')    
    #saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    total_step = 0 # Explore or exploit parameter
    episode_reward = deque(maxlen=100) # 100 episodes for running average/running mean/window

    # Training episodes/epochs
    for ep in range(11111):
        aloss_batch, eloss_batch, aloss2_batch = [], [], []
        total_reward = 0
        state = env.reset()
        num_step = 0
        rate = -1

        # Training steps/batches
        while True:
            # Explore (env) or Exploit (model)
            total_step += 1
            explore_p = explore_stop + (explore_start - explore_stop) * np.exp(-decay_rate * total_step) 
            if explore_p > np.random.rand():
                action = env.action_space.sample()
            else:
                action_logits = sess.run(model.actions_logits, feed_dict={model.states: state.reshape([1, -1])})
                action = np.argmax(action_logits)
            next_state, reward, done, _ = env.step(action)
            memory.buffer.append([state, action, next_state, reward, float(done), rate])
            num_step += 1 # momory added
            total_reward += reward
            state = next_state
            
            # Training with the maxrated minibatch
            batch = memory.buffer
            #for idx in range(memory_size// batch_size):
            while True:
                idx = np.random.choice(np.arange(memory_size// batch_size))
                states = np.array([each[0] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                actions = np.array([each[1] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                next_states = np.array([each[2] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                rewards = np.array([each[3] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                dones = np.array([each[4] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                rates = np.array([each[5] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                states = states[rates >= np.max(rates)]
                actions = actions[rates >= np.max(rates)]
                next_states = next_states[rates >= np.max(rates)]
                rewards = rewards[rates >= np.max(rates)]
                dones = dones[rates >= np.max(rates)]
                rates = rates[rates >= np.max(rates)]
                if np.count_nonzero(dones) > 0 and len(dones) > 1 and np.max(rates) > 0:
                    break
            aloss, _ = sess.run([model.a_loss, model.a_opt],
                                  feed_dict = {model.states: states, 
                                               model.actions: actions,
                                               model.next_states: next_states,
                                               model.rewards: rewards,
                                               model.dones: dones,
                                               model.rates: rates})
            eloss, _ = sess.run([model.e_loss, model.e_opt],
                                  feed_dict = {model.states: states, 
                                               model.actions: actions,
                                               model.next_states: next_states,
                                               model.rewards: rewards,
                                               model.dones: dones,
                                               model.rates: rates})
            aloss2, _= sess.run([model.a_loss2, model.a_opt2], 
                                 feed_dict = {model.states: states, 
                                              model.actions: actions,
                                              model.next_states: next_states,
                                              model.rewards: rewards,
                                              model.dones: dones,
                                              model.rates: rates})
            # print(len(dones), np.count_nonzero(dones), np.max(rates))
            aloss_batch.append(aloss)
            eloss_batch.append(eloss)
            aloss2_batch.append(aloss2)
            if done is True:
                break
                
        # Rating the latest played episode
        rate = total_reward/500 # update rate at the end/ when episode is done
        for idx in range(num_step): # episode length
            if memory.buffer[-1-idx][-1] == -1: # double-check the landmark/marked indexes
                memory.buffer[-1-idx][-1] = rate # rate the trajectory/data

        # Print out
        episode_reward.append(total_reward)
        print('Episode:{}'.format(ep),
              'meanR:{:.4f}'.format(np.mean(episode_reward)),
              'R:{:.4f}'.format(total_reward),
              'rate:{:.4f}'.format(rate),
              'aloss:{:.4f}'.format(np.mean(aloss_batch)),
              'eloss:{:.4f}'.format(np.mean(eloss_batch)),
              'aloss2:{:.4f}'.format(np.mean(aloss2_batch)),
              'exploreP:{:.4f}'.format(explore_p))

        # Ploting out
        episode_rewards_list.append([ep, np.mean(episode_reward)])
        rewards_list.append([ep, total_reward])
        aloss_list.append([ep, np.mean(aloss_batch)])
        eloss_list.append([ep, np.mean(eloss_batch)])
        aloss2_list.append([ep, np.mean(aloss2_batch)])
        
        # Break episode/epoch loop
        ## Option 1: Solve the First Version
        #The task is episodic, and in order to solve the environment, 
        #your agent must get an average score of +30 over 100 consecutive episodes.
        if np.mean(episode_reward) >= 500:
            break
            
    # At the end of all training episodes/epochs
    saver.save(sess, 'checkpoints/model.ckpt')

Episode:0 meanR:11.0000 R:11.0000 rate:0.0220 aloss:0.7243 eloss:3.7025 aloss2:2.5660 exploreP:0.9989
Episode:1 meanR:9.5000 R:8.0000 rate:0.0160 aloss:0.7154 eloss:3.7115 aloss2:2.3260 exploreP:0.9981
Episode:2 meanR:16.6667 R:31.0000 rate:0.0620 aloss:0.7234 eloss:3.6731 aloss2:2.4525 exploreP:0.9951
Episode:3 meanR:22.2500 R:39.0000 rate:0.0780 aloss:0.7143 eloss:3.6474 aloss2:2.3716 exploreP:0.9912
Episode:4 meanR:23.0000 R:26.0000 rate:0.0520 aloss:0.7080 eloss:3.6195 aloss2:2.3406 exploreP:0.9887
Episode:5 meanR:21.8333 R:16.0000 rate:0.0320 aloss:0.7112 eloss:3.6018 aloss2:2.3450 exploreP:0.9871
Episode:6 meanR:26.0000 R:51.0000 rate:0.1020 aloss:0.7062 eloss:3.5395 aloss2:2.3790 exploreP:0.9821
Episode:7 meanR:25.2500 R:20.0000 rate:0.0400 aloss:0.7044 eloss:3.5046 aloss2:2.3351 exploreP:0.9802
Episode:8 meanR:23.8889 R:13.0000 rate:0.0260 aloss:0.7024 eloss:3.4724 aloss2:2.4661 exploreP:0.9789
Episode:9 meanR:23.1000 R:16.0000 rate:0.0320 aloss:0.7032 eloss:3.4768 aloss2:2.237

Episode:80 meanR:21.7654 R:18.0000 rate:0.0360 aloss:0.6992 eloss:2.8317 aloss2:2.0729 exploreP:0.8400
Episode:81 meanR:21.6829 R:15.0000 rate:0.0300 aloss:0.6879 eloss:2.8060 aloss2:2.1086 exploreP:0.8387
Episode:82 meanR:21.6627 R:20.0000 rate:0.0400 aloss:0.6942 eloss:2.7871 aloss2:2.1444 exploreP:0.8371
Episode:83 meanR:21.7381 R:28.0000 rate:0.0560 aloss:0.7029 eloss:2.9240 aloss2:1.8547 exploreP:0.8348
Episode:84 meanR:22.0706 R:50.0000 rate:0.1000 aloss:0.6923 eloss:2.8126 aloss2:2.1242 exploreP:0.8307
Episode:85 meanR:22.0000 R:16.0000 rate:0.0320 aloss:0.6922 eloss:2.8379 aloss2:1.9786 exploreP:0.8293
Episode:86 meanR:21.8851 R:12.0000 rate:0.0240 aloss:0.7107 eloss:2.8022 aloss2:2.1374 exploreP:0.8284
Episode:87 meanR:22.2386 R:53.0000 rate:0.1060 aloss:0.6969 eloss:2.8074 aloss2:2.1260 exploreP:0.8240
Episode:88 meanR:22.2022 R:19.0000 rate:0.0380 aloss:0.6920 eloss:2.7779 aloss2:2.2428 exploreP:0.8225
Episode:89 meanR:22.0889 R:12.0000 rate:0.0240 aloss:0.6926 eloss:2.7450 

Episode:159 meanR:23.5300 R:24.0000 rate:0.0480 aloss:0.6872 eloss:2.5133 aloss2:2.5453 exploreP:0.6914
Episode:160 meanR:23.9100 R:53.0000 rate:0.1060 aloss:0.6928 eloss:2.5231 aloss2:2.5857 exploreP:0.6878
Episode:161 meanR:23.9400 R:28.0000 rate:0.0560 aloss:0.6980 eloss:2.5764 aloss2:2.5000 exploreP:0.6859
Episode:162 meanR:23.9300 R:30.0000 rate:0.0600 aloss:0.6911 eloss:2.5206 aloss2:2.5333 exploreP:0.6839
Episode:163 meanR:23.9400 R:36.0000 rate:0.0720 aloss:0.6907 eloss:2.5219 aloss2:2.5579 exploreP:0.6815
Episode:164 meanR:24.2000 R:48.0000 rate:0.0960 aloss:0.7033 eloss:2.5242 aloss2:2.4754 exploreP:0.6783
Episode:165 meanR:24.2300 R:25.0000 rate:0.0500 aloss:0.7000 eloss:2.5104 aloss2:2.5772 exploreP:0.6766
Episode:166 meanR:24.3500 R:34.0000 rate:0.0680 aloss:0.7003 eloss:2.5104 aloss2:2.4992 exploreP:0.6743
Episode:167 meanR:24.4200 R:22.0000 rate:0.0440 aloss:0.6983 eloss:2.5301 aloss2:2.4441 exploreP:0.6729
Episode:168 meanR:24.4800 R:17.0000 rate:0.0340 aloss:0.6899 elo

Episode:238 meanR:22.8300 R:23.0000 rate:0.0460 aloss:0.7476 eloss:2.5284 aloss2:3.3768 exploreP:0.5889
Episode:239 meanR:22.6600 R:22.0000 rate:0.0440 aloss:0.7740 eloss:2.5005 aloss2:3.3806 exploreP:0.5876
Episode:240 meanR:22.3700 R:14.0000 rate:0.0280 aloss:0.7455 eloss:2.5224 aloss2:3.4887 exploreP:0.5868
Episode:241 meanR:22.1400 R:13.0000 rate:0.0260 aloss:0.7247 eloss:2.5365 aloss2:3.3841 exploreP:0.5861
Episode:242 meanR:22.0100 R:15.0000 rate:0.0300 aloss:0.7698 eloss:2.5002 aloss2:3.4543 exploreP:0.5852
Episode:243 meanR:21.8600 R:10.0000 rate:0.0200 aloss:0.7544 eloss:2.4873 aloss2:3.4539 exploreP:0.5846
Episode:244 meanR:21.7500 R:12.0000 rate:0.0240 aloss:0.7541 eloss:2.5157 aloss2:3.3442 exploreP:0.5839
Episode:245 meanR:21.5800 R:23.0000 rate:0.0460 aloss:0.7553 eloss:2.5245 aloss2:3.3936 exploreP:0.5826
Episode:246 meanR:21.3700 R:29.0000 rate:0.0580 aloss:0.7548 eloss:2.5186 aloss2:3.3932 exploreP:0.5810
Episode:247 meanR:21.2600 R:14.0000 rate:0.0280 aloss:0.7703 elo

Episode:317 meanR:22.4200 R:20.0000 rate:0.0400 aloss:0.6941 eloss:2.5236 aloss2:3.4856 exploreP:0.4904
Episode:318 meanR:22.4000 R:36.0000 rate:0.0720 aloss:0.6999 eloss:2.5442 aloss2:3.4908 exploreP:0.4887
Episode:319 meanR:22.4900 R:24.0000 rate:0.0480 aloss:0.6982 eloss:2.5262 aloss2:3.4449 exploreP:0.4876
Episode:320 meanR:22.1400 R:11.0000 rate:0.0220 aloss:0.7016 eloss:2.5243 aloss2:3.4896 exploreP:0.4870
Episode:321 meanR:22.2900 R:29.0000 rate:0.0580 aloss:0.6907 eloss:2.5651 aloss2:3.3931 exploreP:0.4857
Episode:322 meanR:22.3300 R:18.0000 rate:0.0360 aloss:0.6962 eloss:2.5377 aloss2:3.4278 exploreP:0.4848
Episode:323 meanR:22.4900 R:27.0000 rate:0.0540 aloss:0.6986 eloss:2.5510 aloss2:3.3867 exploreP:0.4835
Episode:324 meanR:22.4500 R:15.0000 rate:0.0300 aloss:0.6910 eloss:2.5722 aloss2:3.3597 exploreP:0.4828
Episode:325 meanR:22.4800 R:14.0000 rate:0.0280 aloss:0.6994 eloss:2.5430 aloss2:3.2891 exploreP:0.4822
Episode:326 meanR:22.4600 R:14.0000 rate:0.0280 aloss:0.6957 elo

...

Episode:1501 meanR:41.6700 R:10.0000 rate:0.0200 aloss:0.6163 eloss:2.2964 aloss2:3.5467 exploreP:0.0193
Episode:1502 meanR:41.8200 R:63.0000 rate:0.1260 aloss:0.6639 eloss:2.2840 aloss2:3.5822 exploreP:0.0193
Episode:1503 meanR:42.0500 R:32.0000 rate:0.0640 aloss:0.7076 eloss:2.2841 aloss2:3.6948 exploreP:0.0192
Episode:1504 meanR:42.0500 R:39.0000 rate:0.0780 aloss:0.6892 eloss:2.2788 aloss2:3.6776 exploreP:0.0192
Episode:1505 meanR:42.6500 R:72.0000 rate:0.1440 aloss:0.6892 eloss:2.2906 aloss2:3.6687 exploreP:0.0191
Episode:1506 meanR:42.5100 R:11.0000 rate:0.0220 aloss:0.7268 eloss:2.3205 aloss2:3.4771 exploreP:0.0191
Episode:1507 meanR:42.5200 R:54.0000 rate:0.1080 aloss:0.7027 eloss:2.3119 aloss2:3.5189 exploreP:0.0191
Episode:1508 meanR:43.2200 R:92.0000 rate:0.1840 aloss:0.7035 eloss:2.2903 aloss2:3.6094 exploreP:0.0190
Episode:1509 meanR:43.6300 R:52.0000 rate:0.1040 aloss:0.6903 eloss:2.2788 aloss2:3.7721 exploreP:0.0189
Episode:1510 meanR:43.2300 R:41.0000 rate:0.0820 aloss:

Episode:1580 meanR:49.8400 R:66.0000 rate:0.1320 aloss:0.7112 eloss:2.2905 aloss2:3.7468 exploreP:0.0162
Episode:1581 meanR:50.5100 R:76.0000 rate:0.1520 aloss:0.7221 eloss:2.3179 aloss2:3.6332 exploreP:0.0162
Episode:1582 meanR:50.2000 R:60.0000 rate:0.1200 aloss:0.6994 eloss:2.3010 aloss2:3.7539 exploreP:0.0161
Episode:1583 meanR:50.1700 R:45.0000 rate:0.0900 aloss:0.6607 eloss:2.2828 aloss2:3.8094 exploreP:0.0161
Episode:1584 meanR:50.1400 R:62.0000 rate:0.1240 aloss:0.6868 eloss:2.2892 aloss2:3.6881 exploreP:0.0161
Episode:1585 meanR:50.3400 R:46.0000 rate:0.0920 aloss:0.6711 eloss:2.2954 aloss2:3.7021 exploreP:0.0160
Episode:1586 meanR:49.8300 R:9.0000 rate:0.0180 aloss:0.8469 eloss:2.2923 aloss2:3.7504 exploreP:0.0160
Episode:1587 meanR:50.3500 R:60.0000 rate:0.1200 aloss:0.6687 eloss:2.3013 aloss2:3.7230 exploreP:0.0160
Episode:1588 meanR:49.9000 R:13.0000 rate:0.0260 aloss:0.7047 eloss:2.3035 aloss2:3.5957 exploreP:0.0160
Episode:1589 meanR:50.1100 R:65.0000 rate:0.1300 aloss:0

Episode:1659 meanR:49.3300 R:25.0000 rate:0.0500 aloss:0.6596 eloss:2.2821 aloss2:3.7030 exploreP:0.0143
Episode:1660 meanR:48.9200 R:55.0000 rate:0.1100 aloss:0.6927 eloss:2.2930 aloss2:3.5812 exploreP:0.0143
Episode:1661 meanR:48.5500 R:51.0000 rate:0.1020 aloss:0.6836 eloss:2.3065 aloss2:3.5351 exploreP:0.0142
Episode:1662 meanR:48.6000 R:50.0000 rate:0.1000 aloss:0.6260 eloss:2.3024 aloss2:3.5694 exploreP:0.0142
Episode:1663 meanR:48.1500 R:52.0000 rate:0.1040 aloss:0.6680 eloss:2.2917 aloss2:3.6350 exploreP:0.0142
Episode:1664 meanR:47.8500 R:15.0000 rate:0.0300 aloss:0.6994 eloss:2.3012 aloss2:3.6092 exploreP:0.0142
Episode:1665 meanR:47.8000 R:9.0000 rate:0.0180 aloss:0.6775 eloss:2.2610 aloss2:3.6392 exploreP:0.0142
Episode:1666 meanR:47.7300 R:50.0000 rate:0.1000 aloss:0.7118 eloss:2.2996 aloss2:3.6031 exploreP:0.0142
Episode:1667 meanR:48.1800 R:59.0000 rate:0.1180 aloss:0.6716 eloss:2.2939 aloss2:3.5752 exploreP:0.0141
Episode:1668 meanR:47.9700 R:27.0000 rate:0.0540 aloss:0

Episode:1737 meanR:56.9300 R:163.0000 rate:0.3260 aloss:0.6489 eloss:2.4093 aloss2:3.3404 exploreP:0.0127
Episode:1738 meanR:58.1800 R:149.0000 rate:0.2980 aloss:0.6274 eloss:2.4162 aloss2:3.3048 exploreP:0.0126
Episode:1739 meanR:58.7400 R:100.0000 rate:0.2000 aloss:0.6390 eloss:2.4151 aloss2:3.3407 exploreP:0.0126
Episode:1740 meanR:59.1600 R:102.0000 rate:0.2040 aloss:0.6613 eloss:2.4112 aloss2:3.3203 exploreP:0.0126
Episode:1741 meanR:60.0700 R:166.0000 rate:0.3320 aloss:0.6644 eloss:2.4269 aloss2:3.2796 exploreP:0.0125
Episode:1742 meanR:61.1600 R:125.0000 rate:0.2500 aloss:0.6611 eloss:2.4393 aloss2:3.1118 exploreP:0.0125
Episode:1743 meanR:61.5300 R:85.0000 rate:0.1700 aloss:0.6896 eloss:2.4494 aloss2:3.1365 exploreP:0.0125
Episode:1744 meanR:62.5400 R:110.0000 rate:0.2200 aloss:0.6682 eloss:2.4570 aloss2:3.1137 exploreP:0.0124
Episode:1745 meanR:62.9000 R:115.0000 rate:0.2300 aloss:0.6722 eloss:2.4724 aloss2:3.0718 exploreP:0.0124
Episode:1746 meanR:63.9100 R:148.0000 rate:0.29

Episode:1814 meanR:148.2300 R:161.0000 rate:0.3220 aloss:0.6668 eloss:2.4228 aloss2:3.7671 exploreP:0.0107
Episode:1815 meanR:149.7400 R:183.0000 rate:0.3660 aloss:0.6469 eloss:2.4265 aloss2:3.8156 exploreP:0.0107
Episode:1816 meanR:151.2700 R:207.0000 rate:0.4140 aloss:0.6536 eloss:2.4249 aloss2:3.7825 exploreP:0.0107
Episode:1817 meanR:152.3000 R:162.0000 rate:0.3240 aloss:0.6595 eloss:2.4169 aloss2:3.8089 exploreP:0.0107
Episode:1818 meanR:153.6300 R:196.0000 rate:0.3920 aloss:0.6576 eloss:2.4173 aloss2:3.8390 exploreP:0.0107
Episode:1819 meanR:154.8900 R:188.0000 rate:0.3760 aloss:0.6495 eloss:2.4241 aloss2:3.8853 exploreP:0.0107
Episode:1820 meanR:156.4900 R:210.0000 rate:0.4200 aloss:0.6595 eloss:2.4245 aloss2:3.8548 exploreP:0.0106
Episode:1821 meanR:157.3200 R:200.0000 rate:0.4000 aloss:0.6452 eloss:2.4020 aloss2:3.8655 exploreP:0.0106
Episode:1822 meanR:158.5700 R:179.0000 rate:0.3580 aloss:0.6651 eloss:2.4134 aloss2:3.8532 exploreP:0.0106
Episode:1823 meanR:159.4400 R:175.000

# Visualizing training

Below I'll plot the total rewards for each episode. I'm plotting the rolling average too, in blue.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

def running_mean(x, N):
    cumsum = np.cumsum(np.insert(x, 0, 0)) 
    return (cumsum[N:] - cumsum[:-N]) / N 

In [ ]:
eps, arr = np.array(episode_rewards_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Total rewards')

In [ ]:
eps, arr = np.array(rewards_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Total rewards')

In [ ]:
eps, arr = np.array(aloss_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Act losses')

In [ ]:
eps, arr = np.array(eloss_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Env losses')

In [ ]:
eps, arr = np.array(aloss2_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Act losses 2')

## Testing

Let's checkout how our trained agent plays the game.

In [21]:
import gym
env = gym.make('CartPole-v0')
env = gym.make('CartPole-v1')

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    saver.restore(sess, 'checkpoints/model.ckpt')    
    #saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    
    # Episodes/epochs
    for _ in range(10):
        state = env.reset()
        total_reward = 0

        # Steps/batches
        while True:
            env.render()
            action_logits = sess.run(model.actions_logits, feed_dict={model.states: state.reshape([1, -1])})
            action = np.argmax(action_logits)
            state, reward, done, _ = env.step(action)
            total_reward += reward
            if done:
                print('total_reward: {}'.format(total_reward))
                break
                
env.close()

INFO:tensorflow:Restoring parameters from checkpoints/model.ckpt
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
